In [ ]:
def plot_centers(data, num_echos, segname, minimaCenters=[], pxAvgCenters=[], weightedPxCenters=[], save_path=""):
    dpi=1000
    for echo in range(num_echos):
        if minimaCenters.any():
            plt.figure()
            plt.imshow(data[:,:,echo], cmap='gray')
            plt.scatter(minimaCenters[:,echo,1],minimaCenters[:,echo,0], c='r', s=0.01)
            filename=segname+str(echo)+"-minimas.png"
            if save_path:
                plt.savefig(save_path+filename, dpi=dpi)
            plt.show()
        if pxAvgCenters.any():
            plt.figure()
            plt.imshow(data[:,:,echo], cmap='gray')
            plt.scatter(np.round(pxAvgCenters[:,echo,0]),np.round(pxAvgCenters[:,echo,1]), c='r', s=0.01)
            filename=segname+str(echo)+"-avgCenters.png"
            if save_path:
                plt.savefig(save_path+filename, dpi=dpi)
            plt.show()
        if weightedPxCenters.any():
            plt.figure()
            plt.imshow(data[:,:,echo], cmap='gray')
            plt.scatter(np.round(weightedPxCenters[:,echo,0]),np.round(weightedPxCenters[:,echo,1]), c='r', s=0.01)
            filename=segname+str(echo)+"-weightedCenters.png"
            if save_path:
                plt.savefig(save_path+filename, dpi=dpi)
            plt.show()

In [2]:
def plot_c2c(c2c_dist,minimas,avgCenters,weightedCenters, num_echos=8, save_path=""):
    # wafer-level center2center distances of each pattern
    num_patterns=len(c2c_dist)+1
    distIdx=[]
    for i in range(num_patterns-1):
        m=num_patterns
        j=i+1
        distIdx.append(m * i + j - ((i + 2) * (i + 1)) // 2)
    # dist(X[i,:], X[j,:]) = dists[m * i + j - ((i + 2) * (i + 1)) // 2], X: m-by-n
    print(distIdx)
    distsEchos=np.empty((len(distIdx),num_echos))
    avgDistsEchos=np.empty_like(distsEchos)
    weightedDistsEchos=np.empty_like(distsEchos)
    for echo in range(num_echos):
        dists=spatial.distance.pdist(minimas[:,echo,:], metric="euclidean")
        dists2=spatial.distance.pdist(avgCenters[:,echo,:], metric="euclidean")
        dists3=spatial.distance.pdist(weightedCenters[:,echo,:], metric="euclidean")
        distsEchos[:,echo]=dists[distIdx]*136
        avgDistsEchos[:,echo]=dists2[distIdx]*136
        weightedDistsEchos[:,echo]=dists3[distIdx]*136


    ## Segment vs echo
    x=np.linspace(5,40,8)
    plt.figure()
    for i in range(len(distIdx)):
        plt.plot(x,(distsEchos-np.reshape(c2c_dist, (-1,1)))[i,:])
#     plt.plot(x,(distsEchos-np.reshape(c2c_dist, (-1,1)))[1,:])
#     plt.plot(x,(distsEchos-np.reshape(c2c_dist, (-1,1)))[2,:])
    # plt.plot(x,(distsEchos-np.reshape(c2c_dist, (-1,1)))[3,:])
    plt.xlabel("echo (ms)")
    # plt.xticks([0, 1])
    plt.yticks([-400, -300, -200, -136, 0, 136, 200, 300, 400])
    plt.ylabel("Difference in minima2center distances mri-wafer (um)")
    plt.ylim([-500, 500])
    plt.legend(["segment1", "segment2", "segment3"])
    plt.grid()
    filename="pxlminima-center2center-echos.pdf"
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path+filename, dpi=dpi)
    plt.show()


    x=np.linspace(5,40,8)
    plt.figure()
    for i in range(len(distIdx)):
        plt.plot(x,(avgDistsEchos-np.reshape(c2c_dist, (-1,1)))[i,:])
#     plt.plot(x,(avgDistsEchos-np.reshape(c2c_dist, (-1,1)))[1,:])
#     plt.plot(x,(avgDistsEchos-np.reshape(c2c_dist, (-1,1)))[2,:])
    # plt.plot(x,(distsEchos-np.reshape(c2c_dist, (-1,1)))[3,:])
    plt.xlabel("echo (ms)")
    # plt.xticks([0, 1])
    plt.yticks([-400, -300, -200, -136, 0, 136, 200, 300, 400])
    plt.ylabel("Difference in pxl averaged center distances mri-wafer (um)")
    plt.ylim([-500, 500])
    plt.legend(["segment1", "segment2", "segment3"])
    plt.grid()
    filename="averaged-center2center-echos.pdf"
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path+filename, dpi=dpi)
    plt.show()



    x=np.linspace(5,40,8)
    plt.figure()
    for i in range(len(distIdx)):
        plt.plot(x,(weightedDistsEchos-np.reshape(c2c_dist, (-1,1)))[i,:])
#     plt.plot(x,(weightedDistsEchos-np.reshape(c2c_dist, (-1,1)))[1,:])
#     plt.plot(x,(weightedDistsEchos-np.reshape(c2c_dist, (-1,1)))[2,:])
    # plt.plot(x,(distsEchos-np.reshape(c2c_dist, (-1,1)))[3,:])
    plt.xlabel("echo (ms)")
    # plt.xticks([0, 1])
    plt.yticks([-400, -300, -200, -136, 0, 136, 200, 300, 400])
    plt.ylabel("Difference in weighted-center distances mri-wafer (um)")
    plt.ylim([-500, 500])
    plt.legend(["segment1", "segment2", "segment3"])
    plt.grid()
    filename="weighted-center2center-echos.pdf"
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path+filename, dpi=dpi)
    plt.show()


In [4]:
def plot_roi(img, structs_idx, roi_name, seg, echo, save_path, savefigs=False, dpi=1000):
    colors=["r", "b", "g", "m", "o"]
    legends=[]
    plt.figure()
    plt.imshow(img, cmap='gray')
    for i, key in enumerate(structs_idx.keys()):
        if structs_idx[key]:
            plt.scatter(np.array(structs_idx[key])[:,1], np.array(structs_idx[key])[:,0], marker='o', c=colors[i], s=0.05, edgecolor='none')
    if savefigs:
        filename=roi_name+str(seg)+"echo"+str(echo)+".png"
        plt.savefig(save_path+"roi/"+filename, dpi=dpi)
    plt.show()